In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
week6_link = 'http://www.footballlocks.com/nfl_odds_week_6.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week6_html = requests.get(week6_link).text

soup = BeautifulSoup(week6_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[270])

<tr>
<td>10/11 8:20 ET</td>
<td>Philadelphia</td>
<td>-2</td>
<td>At NY Giants</td>
<td>45</td>
<td>-$130 +$110</td>
</tr>
<tr>
<td>10/16 8:30 ET</td>
<td>Chicago</td>
<td>-12.5</td>
<td>At Arizona</td>
<td>40.5</td>
<td>-$850 +$650</td>
</tr>


In [4]:
#want to 271 because it does not include last one
table = tr[51:271]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week6_df = pd.DataFrame(string_table)

week6_df.columns = ['all_data']

week6_df.head()

,all_data
0,<tr>\n<td>10/11 8:20 ET</td>\n<td>Philadelphia...
1,<tr>\n<td>10/14 1:00 ET</td>\n<td>At Minnesota...
2,<tr>\n<td>10/14 1:00 ET</td>\n<td>At Cleveland...
3,<tr>\n<td>10/14 1:00 ET</td>\n<td>Chicago</td>...
4,<tr>\n<td>10/14 1:00 ET</td>\n<td>Carolina</td...


In [5]:
#want to clean the date data

dates = [x.split('td')[1] for x in week6_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week6_df['date'] = cleaned_dates

In [11]:
#pull out favorites

week6_df['cleaned_favorites'] = week6_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week6_df['cleaned_underdogs'] = week6_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week6_df['cleaned_spread'] = week6_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week6_df['cleaned_over_under'] = week6_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week6_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week6_df['money_odds'] = money_odds

#input week number for each game
week6_df['week_number'] = '6'

In [12]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week6_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week6_df['year'] = year_list

In [13]:
#create a final dataframe for week 4
week6_final = week6_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week6_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,10/11 8:20 ET,2018,6,Philadelphia,At NY Giants,-2,45,-$130 +$110
1,10/14 1:00 ET,2018,6,At Minnesota,Arizona,-9.5,43.5,-$500 +$375
2,10/14 1:00 ET,2018,6,At Cleveland,LA Chargers,-1,47,-$120 +$100
3,10/14 1:00 ET,2018,6,Chicago,At Miami,-7,40.5,-$340 +$280
4,10/14 1:00 ET,2018,6,Carolina,At Washington,-1,44.5,-$120 +$100


In [15]:
#week6_final to pickle
week6_final.to_pickle('week6_final.pkl')